In [18]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast


In [19]:
from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirect, ForecasterDirectMultiVariate
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [20]:
import numpy as np
import pandas as pd

def is_view(obj1, obj2):
    """
    Check if obj2 is a view of obj1.
    Works for NumPy arrays, pandas Series, and pandas DataFrames.
    """
    # Convert pandas objects to underlying NumPy arrays
    if isinstance(obj1, (pd.Series, pd.DataFrame)):
        arr1 = obj1.values
    else:
        arr1 = obj1

    if isinstance(obj2, (pd.Series, pd.DataFrame)):
        arr2 = obj2.values
    else:
        arr2 = obj2

    # Use NumPy function to check shared memory
    return np.shares_memory(arr1, arr2)

In [21]:
rng = np.random.default_rng(123)
y = pd.Series(rng.normal(size=10000), index=pd.date_range("2020-01-01", periods=10000, freq="h"))
y_original = y.copy()
y_values = y.to_numpy()
forecaster = ForecasterRecursive(
    regressor=LinearRegression(),
    lags=100
)

In [22]:
x_train, y_train = forecaster._create_lags(y=y_values)
x_train_old, y_train_old = forecaster._create_lags_deprecated(y=y_values)
np.testing.assert_array_equal(x_train, x_train_old)
np.testing.assert_array_equal(y_train, y_train_old)

In [28]:
np.shares_memory(x_train, y_values)

False

In [5]:
%%timeit
forecaster._create_lags_deprecated(
    y
)

5.51 ms ± 385 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%timeit
forecaster._create_lags(
    y
)

827 μs ± 47.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
x_train, y_train = forecaster.create_train_X_y(y=y)
x_train.iloc[1,1] = 10000
pd.testing.assert_series_equal(y, y_original)

In [8]:
%%timeit
forecaster.fit(y=y)

234 ms ± 36.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
rng = np.random.default_rng(123)
series = {
    f"series_{i}": pd.Series(
        rng.normal(size=1000), index=pd.date_range("2020-01-01", periods=1000)
    )
    for i in range(1000)
}
forecaster = ForecasterRecursiveMultiSeries(
    regressor=LinearRegression(),
    lags=3,
)

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : skforecast.exceptions.DataTransformationWarning                           │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive │
│ _multiseries.py:487                                                                  │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [10]:
for y in series.values():
    x_train, y_train = forecaster._create_lags(y=y.to_numpy())
    x_train_old, y_train_old = forecaster._create_lags_deprecated(y=y.to_numpy())
    np.testing.assert_array_equal(x_train, x_train_old)
    np.testing.assert_array_equal(y_train, y_train_old)

In [11]:
%%timeit

for y in series.values():
    forecaster._create_lags_deprecated(
        y
    )

147 ms ± 4.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit

for y in series.values():
    forecaster._create_lags(
        y
    )

78.9 ms ± 2.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit

forecaster.fit(series=series)

2.69 s ± 109 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
from skforecast.direct import ForecasterDirect

forecaster = ForecasterDirect(
    regressor=LinearRegression(),
    steps=24,
    lags=50
)
rng = np.random.default_rng(123)
y = pd.Series(rng.normal(size=1000), index=pd.date_range("2020-01-01", periods=1000))

In [15]:
x_train, y_train = forecaster._create_lags(y=y.to_numpy())
x_train_old, y_train_old = forecaster._create_lags_deprecated(y=y.to_numpy())
np.testing.assert_array_equal(x_train, x_train_old)
np.testing.assert_array_equal(y_train, y_train_old)

In [16]:
%%timeit
forecaster._create_lags_deprecated(
    y
)

2.59 ms ± 23.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit
forecaster._create_lags(
    y
)

64.6 μs ± 3.76 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
%%timeit
forecaster.fit(y=y)

182 ms ± 13.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
forecaster = ForecasterDirectMultiVariate(
    regressor=LinearRegression(),
    steps=24,
    lags=50,
    level = "series_1"
)
rng = np.random.default_rng(123)
series = {
    f"series_{i}": pd.Series(
        rng.normal(size=1000), index=pd.date_range("2020-01-01", periods=1000)
    )
    for i in range(20)
}
series = pd.DataFrame(series)

In [29]:
%%timeit
forecaster._create_train_X_y(
    series = series,
)

14.3 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
forecaster.fit(series=series)

TypeError: ForecasterDirectMultiVariate._create_lags() missing 1 required positional argument: 'lags'

In [ ]:
%%timeit
forecaster.fit(series=series)